In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d5945930ef9b5a2f0811b3eb58439380c4f7be4339c96978e6c9b73a98a20c7c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:


my_spark_session = SparkSession.builder.getOrCreate()
my_spark_context = my_spark_session.sparkContext


In [4]:
path_to_data = "/content/q4/data/friendships.txt"
raw_data = my_spark_context.textFile(path_to_data)


In [5]:
def process_line(line):
    parts = line.split('\t')
    if len(parts) == 1 or parts[0] == '':
        return []

    user_id = int(parts[0])
    friends_list = parts[1].split(',')

    direct_friends = []
    mutual_friend_suggestions = []

    for friend in friends_list:
        if friend != '':
            direct_friends.append(((user_id, int(friend)), 0))

    for i in range(len(friends_list) - 1):
        for j in range(i + 1, len(friends_list)):
            mutual_friend_suggestions.append(((int(friends_list[i]), int(friends_list[j])), 1))
            mutual_friend_suggestions.append(((int(friends_list[j]), int(friends_list[i])), 1))

    return direct_friends + mutual_friend_suggestions


In [6]:
friend_pairs = raw_data.flatMap(lambda line: process_line(line))

In [10]:
existing_friends = friend_pairs.filter(lambda relationship: relationship[1] == 0)

In [11]:
mutual_friends = friend_pairs.subtractByKey(existing_friends).reduceByKey(lambda a, b: a + b)
mutual_friends_grouped = mutual_friends.map(lambda x: (x[0][0], (x[0][1], x[1]))).groupByKey().mapValues(list)

In [13]:
def suggest_friends(user_and_potential_friends, top_n=10):
    user, potential_friends_list = user_and_potential_friends

    sorted_potential_friends = sorted(potential_friends_list, key=lambda friend_info: (-friend_info[1], friend_info[0]))[:top_n]

    recommendations = [friend for friend, mutual_count in sorted_potential_friends]

    return user, recommendations


In [14]:
final_suggestions = mutual_friends_grouped.map(lambda user_and_friends: suggest_friends(user_and_friends))
formatted_results = final_suggestions.map(lambda user_recs: "{}\t{}".format(user_recs[0], ",".join(map(lambda x: str(x), user_recs[1])))).collect()


In [16]:
target_user_ids = ['11']
for target_user_id in target_user_ids:
    for line in formatted_results:
        user, recommendations = line.split('\t')
        if user == target_user_id:
            print(line)

11	27552,7785,27573,27574,27589,27590,27600,27617,27620,27667
